In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
## Calculate summed MSEs for all timesteps for selection of alphas

# Get current usage
usage = Image.open('../assets/wales_current_welsh_usage.tif')
usage = np.array(usage) * 0.01
usage = np.flip(usage, axis=0)

usage_mask = Image.open("../assets/wales_current_usage_mask.tif")
usage_mask = np.array(usage_mask).astype(bool)
usage_mask = np.flip(usage_mask, axis=0)
usage[~usage_mask] = np.nan

freq_mask = Image.open("../assets/attempt2_wales_country_mask.tif")
freq_mask = np.array(freq_mask).astype(bool)
freq_mask = np.flip(freq_mask, axis=0)
mask_shape = freq_mask.shape

total_diffs = []
ax_num = 0
all_nums = []
alphas = [1.2, 1.7, 2.3, 2.5]

print("started")
for alpha in alphas:
    
    # Load data and get timestamps from filenames
    names = os.listdir(f'../simulation_data/walesICbook5Alpha{alpha}Beta1.1SigmavarFactor2Deltat0.0004Tmax500.0_0')
    nums = [name.split('_')[-1].replace('.npy','') for name in names]
    nums = np.array(nums).astype(int)
    nums = np.array(sorted(nums))[:len(nums)//2]
    ks=nums
    all_nums.append(nums)
    
    # Calculate summed MSE
    alpha_diffs = []
    for i, k in enumerate(ks):
        print(f'{i} out of {len(nums)} completed', end = '\r')

        freq = np.load(f"../simulation_data/walesICbook5Alpha{alpha}Beta1.1SigmavarFactor2Deltat0.0004Tmax500.0_0/walesICbook5Alpha{alpha}Beta1.1SigmavarFactor2Deltat0.0004Tmax500.0MEMORY_0_{k}.npy")
        freq = freq[:mask_shape[0], :mask_shape[1]]
        freq[~freq_mask] = np.nan

        usage = usage/(np.nanmax(usage))
        diff = (usage - freq)**2
        total_diff = np.nansum(diff)
        alpha_diffs.append(total_diff)

    total_diffs.append(alpha_diffs)

In [ ]:
# Plot diffs for the alphas used

plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

plt.figure(figsize=(4,4))

for i, alpha in enumerate(alphas):
    plt.plot(all_nums[i], total_diffs[i], marker=".", label=alphas[i])

plt.ylabel(r"$(f_\textrm{actual} - f_\textrm{pred})^2$")
plt.xlim(0,300000)
plt.legend()
plt.show()

In [ ]:
# Get index of min MSE^2 (alpha=2.5 is non-physical)

min_ind = np.argmin(total_diffs[2])
min_ind_timestamp = all_nums[2][min_ind]
min_ind_timestamp

In [ ]:
usage = Image.open('wales_current_welsh_usage.tif')
usage = np.array(usage) * 0.01
usage = np.flip(usage, axis=0)

freq_mask = Image.open("attempt2_wales_country_mask.tif")
freq_mask = np.array(freq_mask).astype(bool)
freq_mask = np.flip(freq_mask, axis=0)
mask_shape = freq_mask.shape

k=15000
freq = np.load(f"../simulation_data/walesICbook5Alpha2.3Beta1.1SigmavarFactor2Deltat0.0004Tmax500.0_0/walesICbook5Alpha2.3Beta1.1SigmavarFactor2Deltat0.0004Tmax500.0MEMORY_0_{k}.npy")
    
freq = freq[:mask_shape[0], :mask_shape[1]]
freq[~freq_mask] = np.nan

diff = usage - freq
total_diff = np.nansum(diff)

# Plot 'residuals'
fig, ax = plt.subplots()
plt.imshow(diff, origin="lower", cmap='bwr', vmin=-1.0, vmax=1.0)
cb = plt.colorbar()
cb.set_label("Actual $-$ predicted use", rotation=270, labelpad=15)
ax.axis("off")
s = fig.get_size_inches()
fig.set_size_inches(float(s[0])*0.6, float(s[1])*0.6)
plt.savefig("wales_residuals_2.3_15000.pdf", bbox_inches="tight")
plt.show()
